In [111]:
import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt
import math
import re
import datetime
from shapely import wkt
import urllib
import json
import csv
import requests
import jdc
import geopandas as gpd

THE FOLLOWING IS THE ACTUAL SCHEMA, WHICH WILL BE DEFINED AS A SINGLE DEFINITION

In [157]:
def tryParseWKT(x):
    try:
        return wkt.loads(x)
    except:
        return None

def geometry(permits):
    geom_count = 0
    
    for col in permits.columns:
      if re.search('latitude|longitude|pointx|pointy', col, re.IGNORECASE):
        geom_count = 0
        break
      else:
        permits[col] = permits[col].astype(str)
        if permits[col].str.contains('POINT \(').any():
          geom_count += 1
          geo = col
        else: continue
    
    if geom_count > 1:
      raise ValueError('More than one POINT geometry column. Investigate.')

    elif geom_count == 1:
      permits = permits[permits[geo].notna()]
      permits['geometry'] = permits[geo]
    
      permits['geometry'] = permits['geometry'].replace('nan',None)
      permits['geometry'] = permits['geometry'].apply(lambda x: tryParseWKT(x))
    
      permits = gpd.GeoDataFrame(permits, geometry = permits['geometry'], crs=4326)
      permits['Longitude'] = permits['geometry'].x
      permits['Latitude'] = permits['geometry'].y

    elif geom_count <= 0:
      permits = permits 
  
    return permits.head(1)

In [117]:
austin = pd.DataFrame()
chicago = pd.DataFrame()
sanfran = pd.DataFrame()
nyc = pd.DataFrame()

urls = [
    'https://data.austintexas.gov/resource/3syk-w9eu.csv?$limit=5000&$order=applieddate%20DESC&$where=' + urllib.parse.quote('applieddate IS NOT NULL'),
    'https://data.cityofchicago.org/resource/ydr8-5enu.csv?$limit=5000&$order=application_start_date%20DESC&$where=' + urllib.parse.quote('application_start_date IS NOT NULL'),
    'https://data.sfgov.org/resource/i98e-djp9.csv?$limit=500&$order=permit_creation_date%20DESC&$where=' + urllib.parse.quote('permit_creation_date IS NOT NULL'),
    'https://data.cityofnewyork.us/resource/ipu4-2q9a.csv?$limit=500&$order=filing_date%20DESC&$where=' + urllib.parse.quote('filing_date IS NOT NULL'),
]

for url in urls:
  if re.search('data.austintexas.gov', url):
    austin = pd.read_csv(url)
    austin.name = 'Austin'
  elif re.search('data.cityofchicago.org', url):
    chicago = pd.read_csv(url)
    chicago.name = 'Chicago'
  elif re.search('data.sfgov.org', url):
    sanfran = pd.read_csv(url)
    sanfran.name = 'San Francisco'
  elif re.search('data.cityofnewyork.us', url):
    nyc = pd.read_csv(url)
    nyc.name = 'New York City'

In [153]:
permits[permits.work_description.str.contains('POINT \(')].work_description

475     TUCKPOINT (2000 SQ FT) ON EAST ELEVATION (no o...
1256    TUCKPOINT (APPX 400 SQ FT) (NO STRUCTURAL WORK...
3527    TUCKPOINT (APPX 500 SQ FT) AND LINTEL REPLACEM...
Name: work_description, dtype: object

In [158]:
permits = chicago

geometry(permits)

,id,permit_,permit_type,review_type,application_start_date,issue_date,processing_time,street_number,street_direction,street_name,...,pin9,pin10,community_area,census_tract,ward,xcoordinate,ycoordinate,latitude,longitude,location
0,3330765,101021961,PERMIT - EASY PERMIT PROCESS,EASY PERMIT WEB,2023-06-26T00:00:00.000,2023-06-26T00:00:00.000,0,5436,W,SCHOOL,...,nan,nan,15.0,151100.0,30.0,1139376.66557001,1921444.2054039924,41.940556,-87.763153,POINT (-87.763153195317 41.940555558327)
